A notebook to test a DPT model.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
# import numpy as np
from natsort import natsorted

from train_dpt import DPTSolver
from notebooks.utils import *

Download a model from a checkpoint. The last checkpoint is used.

In [ ]:
def get_checkpoint(run_name):
    read_dir = os.path.join("../../results", "DPT_3", run_name, "checkpoints")
    checkpoint = natsorted(os.listdir(read_dir))[-1]
    checkpoint = os.path.join(read_dir, checkpoint)
    return checkpoint

In [ ]:
device = "cuda:0"
run_name = "bitflip_gt_ce"
model_name = "DPT"
checkpoint = get_checkpoint(run_name)
model = DPTSolver.load_from_checkpoint(checkpoint).to(device)

Define a budget.

In [ ]:
budget = 5 * model.config["model_params"]["seq_len"]

Run the model in an online inference mode for a given problem.

In [ ]:
read_dir = "../data/normal"
problem_list = natsorted(os.listdir(read_dir))

model_results = defaultdict(dict)
for problem in problem_list:
    model_results[problem] = run_model(model, read_dir, problem, name=model_name, budget=budget)

Save the results.

In [ ]:
save_dir = '../final_results/normal'
os.makedirs(save_dir, exist_ok=True)
save_name = f'{model_name}_{run_name}_1k'
save_path = os.path.join(save_dir, save_name + '.npy')
np.save(save_path, model_results)

Split the results according to the problems. In distribution problems are those we used to train the model on, and out of distribution problems are those the model did't see during training.

In [ ]:
exclude_problems = model.config["exclude_problems"]
include_problems = [problem for problem in problem_list if problem not in exclude_problems]

scaled_model_results = scale_meta_dict(model_results)
model_id_results = {k: scaled_model_results[k] for k in include_problems}
model_ood_results = {k: scaled_model_results[k] for k in exclude_problems}

Average the results over the problems to get a common trajectory.

In [ ]:
id_name = 'In distribution problems'
ood_name = 'Out of distribution problems'
model_results_averaged = {
    id_name: get_problem_averaged_meta_dict(model_id_results),
    ood_name: get_problem_averaged_meta_dict(model_ood_results)
}

Show the results. Here one can go to `utils.py` file and adjust the printing settings. Namely, it is possible to move the legend or turn on clipping.

In [ ]:
show_meta_results(model_results_averaged)

Load the other solvers results and split them according to the problems. Then average them over the problems as well.

In [ ]:
read_dir = "../results/normal"
solver_list = ('PSO', 'PROTES', 'GUROBI')

solver_results = defaultdict(dict)
for problem in problem_list:
    for solver in solver_list:
        solver_results[problem][solver] = get_meta_results(problem, solver, read_dir, suffix='test', budget=budget, n_steps=100)

In [ ]:
save_name = 'solvers'
save_path = os.path.join(save_dir, save_name + '.npy')
# if not os.path.exists(save_path):
np.save(save_path, solver_results)

Plot all the results for comparison.

In [ ]:
best_model = 'DPT (warmup) (sample)'
results = {
    problem: {'DPT': model_results[problem][best_model]} | solver_results[problem] 
    for problem in problem_list
}
scaled_results = scale_meta_dict(results)

id_results = {k: scaled_results[k] for k in include_problems}
ood_results = {k: scaled_results[k] for k in exclude_problems}

results_averaged = {
    id_name: get_problem_averaged_meta_dict(id_results),
    ood_name: get_problem_averaged_meta_dict(ood_results)
}

In [ ]:
show_meta_results(results)

In [ ]:
show_meta_results(results_averaged)

Stuff

In [ ]:
# warmup = 0
# model.config["temperature"] = lambda x: math.sqrt(x)
# model.config["temperature"] = lambda x: 5 - 4 * x
# model.config["temperature"] = lambda x: 1 / math.sqrt(1 + x)